In [1]:
"""
TOPSIS Analysis for Text Generation Models
==========================================
This script ranks text generation models using TOPSIS
(Technique for Order of Preference by Similarity to Ideal Solution).

Models evaluated:
GPT-2, GPT-Neo, T5, BART, LLaMA-2, Mistral

Criteria:
Quality → BLEU, ROUGE-L, BERTScore
Efficiency → Latency, VRAM usage, Model size
"""

import numpy as np
import pandas as pd
from tabulate import tabulate
import os

# Create results directory
os.makedirs('results', exist_ok=True)

# --------------------------------------------------
# MODELS
# --------------------------------------------------
models = [
    'GPT-2',
    'GPT-Neo-1.3B',
    'T5-base',
    'BART-large',
    'LLaMA-2-7B',
    'Mistral-7B'
]

# --------------------------------------------------
# DECISION MATRIX
# (Example benchmark-based values)
# --------------------------------------------------
# Columns:
# BLEU, ROUGE-L, BERTScore, Latency(ms), VRAM(GB), Model Size(MB)

data = np.array([
    [28.5, 31.2, 0.86, 40, 0.5, 500],    # GPT-2
    [32.8, 35.6, 0.89, 85, 1.3, 2600],   # GPT-Neo
    [36.5, 38.1, 0.91, 72, 0.9, 850],    # T5-base
    [38.9, 40.3, 0.93, 95, 1.6, 1600],   # BART-large
    [41.2, 42.5, 0.95, 120, 3.5, 4200],  # LLaMA-2
    [42.8, 44.1, 0.96, 110, 3.0, 4100]   # Mistral
])

criteria = [
    'BLEU Score',
    'ROUGE-L',
    'BERTScore',
    'Latency (ms)',
    'VRAM Usage (GB)',
    'Model Size (MB)'
]

# True = higher is better
# False = lower is better
beneficial = [True, True, True, False, False, False]

# Weights must sum to 1
weights = np.array([0.25, 0.20, 0.20, 0.15, 0.10, 0.10])

# --------------------------------------------------
print("="*80)
print("TOPSIS ANALYSIS FOR TEXT GENERATION MODELS")
print("="*80)
print()

# Display decision matrix
df = pd.DataFrame(data, index=models, columns=criteria)
print("Input Decision Matrix:")
print(tabulate(df, headers='keys', tablefmt='grid', floatfmt='.3f'))
print()

# --------------------------------------------------
# STEP 1: NORMALIZE
# --------------------------------------------------
print("Step 1: Normalizing matrix...\n")

norm_divisors = np.sqrt(np.sum(data**2, axis=0))
normalized_data = data / norm_divisors

df_norm = pd.DataFrame(normalized_data, index=models, columns=criteria)
print(tabulate(df_norm, headers='keys', tablefmt='grid', floatfmt='.4f'))
print()

# --------------------------------------------------
# STEP 2: WEIGHTED NORMALIZED MATRIX
# --------------------------------------------------
print("Step 2: Applying weights...\n")

weighted_data = normalized_data * weights

df_weighted = pd.DataFrame(weighted_data, index=models, columns=criteria)
print(tabulate(df_weighted, headers='keys', tablefmt='grid', floatfmt='.4f'))
print()

# --------------------------------------------------
# STEP 3: IDEAL BEST & WORST
# --------------------------------------------------
print("Step 3: Finding ideal best and worst...\n")

ideal_best = np.zeros(len(criteria))
ideal_worst = np.zeros(len(criteria))

for i in range(len(criteria)):
    if beneficial[i]:
        ideal_best[i] = np.max(weighted_data[:, i])
        ideal_worst[i] = np.min(weighted_data[:, i])
    else:
        ideal_best[i] = np.min(weighted_data[:, i])
        ideal_worst[i] = np.max(weighted_data[:, i])

print("Ideal Best (A+):", ideal_best)
print("Ideal Worst (A-):", ideal_worst)
print()

# --------------------------------------------------
# STEP 4: DISTANCE FROM IDEAL
# --------------------------------------------------
print("Step 4: Calculating separation measures...\n")

dist_best = np.sqrt(np.sum((weighted_data - ideal_best)**2, axis=1))
dist_worst = np.sqrt(np.sum((weighted_data - ideal_worst)**2, axis=1))

sep_df = pd.DataFrame({
    "Model": models,
    "Distance from Best": dist_best,
    "Distance from Worst": dist_worst
})
print(tabulate(sep_df, headers='keys', tablefmt='grid', floatfmt='.6f', showindex=False))
print()

# --------------------------------------------------
# STEP 5: TOPSIS SCORE
# --------------------------------------------------
print("Step 5: Computing TOPSIS scores...\n")

scores = dist_worst / (dist_best + dist_worst)

results = pd.DataFrame({
    "Model": models,
    "TOPSIS Score": scores,
    "Rank": np.argsort(-scores) + 1
}).sort_values("Rank")

print(tabulate(results, headers='keys', tablefmt='grid', floatfmt='.6f', showindex=False))
print()

# --------------------------------------------------
# FINAL RANKING
# --------------------------------------------------
print("="*80)
print("FINAL RANKING")
print("="*80)

for _, row in results.iterrows():
    print(f"{int(row['Rank'])}. {row['Model']:<15} Score: {row['TOPSIS Score']:.6f}")

best_model = results.iloc[0]['Model']
print("\nRECOMMENDED MODEL:", best_model)

# --------------------------------------------------
# EXPORT RESULTS
# --------------------------------------------------
results.to_csv("results/topsis_ranking_generation.csv", index=False)

breakdown = pd.DataFrame({
    "Model": models,
    "Distance_from_Best": dist_best,
    "Distance_from_Worst": dist_worst,
    "TOPSIS_Score": scores,
    "Rank": np.argsort(-scores) + 1
})
breakdown.to_csv("results/topsis_breakdown_generation.csv", index=False)

print("\nResults saved in 'results/' folder.")
print("Script completed successfully!")

TOPSIS ANALYSIS FOR TEXT GENERATION MODELS

Input Decision Matrix:
+--------------+--------------+-----------+-------------+----------------+-------------------+-------------------+
|              |   BLEU Score |   ROUGE-L |   BERTScore |   Latency (ms) |   VRAM Usage (GB) |   Model Size (MB) |
+==============+==============+===========+=============+================+===================+===================+
| GPT-2        |       28.500 |    31.200 |       0.860 |         40.000 |             0.500 |           500.000 |
+--------------+--------------+-----------+-------------+----------------+-------------------+-------------------+
| GPT-Neo-1.3B |       32.800 |    35.600 |       0.890 |         85.000 |             1.300 |          2600.000 |
+--------------+--------------+-----------+-------------+----------------+-------------------+-------------------+
| T5-base      |       36.500 |    38.100 |       0.910 |         72.000 |             0.900 |           850.000 |
+------------

In [2]:
"""
Visualization Script for TOPSIS Text Generation Analysis
========================================================
Generates bar charts, radar charts, heatmaps, and ranking visuals.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi
import os

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

os.makedirs('results', exist_ok=True)

# --------------------------------------------------
# MODELS & METRICS
# --------------------------------------------------
models = [
    'GPT-2',
    'GPT-Neo-1.3B',
    'T5-base',
    'BART-large',
    'LLaMA-2-7B',
    'Mistral-7B'
]

criteria = [
    'BLEU Score',
    'ROUGE-L',
    'BERTScore',
    'Latency (ms)',
    'VRAM Usage (GB)',
    'Model Size (MB)'
]

data = np.array([
    [28.5, 31.2, 0.86, 40, 0.5, 500],
    [32.8, 35.6, 0.89, 85, 1.3, 2600],
    [36.5, 38.1, 0.91, 72, 0.9, 850],
    [38.9, 40.3, 0.93, 95, 1.6, 1600],
    [41.2, 42.5, 0.95, 120, 3.5, 4200],
    [42.8, 44.1, 0.96, 110, 3.0, 4100]
])

# Load TOPSIS results from previous script
results_df = pd.read_csv('results/topsis_ranking_generation.csv')
topsis_scores = results_df['TOPSIS Score'].values
ranks = results_df['Rank'].values

print("Generating visualizations...")

# --------------------------------------------------
# 1. TOPSIS SCORE BAR CHART
# --------------------------------------------------
fig, ax = plt.subplots(figsize=(12,6))
bars = ax.barh(models, topsis_scores)

for bar, score in zip(bars, topsis_scores):
    ax.text(bar.get_width()+0.005,
            bar.get_y()+bar.get_height()/2,
            f'{score:.4f}', va='center')

ax.set_title('TOPSIS Score Comparison (Text Generation Models)', fontweight='bold')
ax.set_xlabel('TOPSIS Score')
plt.tight_layout()
plt.savefig('results/topsis_scores_generation.png', dpi=300)
plt.close()

# --------------------------------------------------
# 2. METRIC COMPARISON
# --------------------------------------------------
fig, axes = plt.subplots(2,3, figsize=(16,10))
axes = axes.flatten()

for i, criterion in enumerate(criteria):
    ax = axes[i]
    bars = ax.bar(models, data[:, i])
    ax.set_title(criterion, fontweight='bold')
    ax.tick_params(axis='x', rotation=45)

plt.suptitle('Metrics Comparison Across Models', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('results/metrics_comparison_generation.png', dpi=300)
plt.close()

# --------------------------------------------------
# 3. RADAR CHART
# --------------------------------------------------
radar_data = np.zeros_like(data, dtype=float)

# Normalize (invert cost metrics)
cost_indices = [3,4,5]

for i in range(data.shape[1]):
    if i in cost_indices:
        radar_data[:, i] = 1 - (data[:, i] - data[:, i].min()) / (data[:, i].max() - data[:, i].min())
    else:
        radar_data[:, i] = (data[:, i] - data[:, i].min()) / (data[:, i].max() - data[:, i].min())

angles = [n/float(len(criteria))*2*pi for n in range(len(criteria))]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10,10), subplot_kw=dict(polar=True))

for idx, model in enumerate(models):
    values = radar_data[idx].tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, label=model)
    ax.fill(angles, values, alpha=0.1)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(criteria)
ax.set_title('Normalized Performance Radar Chart', fontweight='bold')
ax.legend(bbox_to_anchor=(1.3,1.1))
plt.tight_layout()
plt.savefig('results/radar_generation.png', dpi=300)
plt.close()

# --------------------------------------------------
# 4. HEATMAP
# --------------------------------------------------
norm_divisors = np.sqrt(np.sum(data**2, axis=0))
normalized_data = data / norm_divisors

plt.figure(figsize=(10,8))
sns.heatmap(normalized_data, annot=True, cmap='YlOrRd',
            xticklabels=criteria, yticklabels=models)

plt.title('Normalized Decision Matrix Heatmap', fontweight='bold')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('results/heatmap_generation.png', dpi=300)
plt.close()

# --------------------------------------------------
# 5. FINAL RANKING CHART
# --------------------------------------------------
sorted_idx = np.argsort(ranks)
sorted_models = [models[i] for i in sorted_idx]
sorted_scores = topsis_scores[sorted_idx]

fig, ax = plt.subplots(figsize=(12,7))
bars = ax.barh(sorted_models, sorted_scores)

for bar, score in zip(bars, sorted_scores):
    ax.text(bar.get_width()+0.005,
            bar.get_y()+bar.get_height()/2,
            f'{score:.4f}', va='center')

ax.set_title('Final Ranking (TOPSIS)', fontweight='bold')
plt.tight_layout()
plt.savefig('results/final_ranking_generation.png', dpi=300)
plt.close()

print("\nAll visualizations created successfully!")
print("Check the results/ folder.")

Generating visualizations...

All visualizations created successfully!
Check the results/ folder.
